***GENERATED CODE FOR rrxgbdsmotephaniallranks PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'EncounterID', 'transformation_label': 'String Indexer'}], 'feature': 'EncounterID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '0001A61A-F158-4D97-85D4-E547EE784C45', 'max': 'FFD4FF35-FCA5-4FB3-BE27-2595F30416F7', 'missing': '0', 'distinct': '485'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'EncounterID'}, {'feature_label': 'EncounterID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('EncounterID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'TransactionDate', 'transformation_label': 'Extract Date'}], 'feature': 'TransactionDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'TransactionDate'}, {'feature_label': 'TransactionDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('TransactionDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateofService', 'transformation_label': 'Extract Date'}], 'feature': 'DateofService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateofService'}, {'feature_label': 'DateofService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateofService')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'}], 'feature': 'ReasonDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ReasonDate'}, {'feature_label': 'ReasonDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ReasonDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BilledPayer', 'transformation_label': 'String Indexer'}], 'feature': 'BilledPayer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '057B6691-F37E-4254-BCE1-D725BDCCD39C', 'max': 'FC574DD7-0F1E-4350-A8F8-A5F7CB490D60', 'missing': '0', 'distinct': '49'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BilledPayer'}, {'feature_label': 'BilledPayer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BilledPayer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DeniedPayer', 'transformation_label': 'String Indexer'}], 'feature': 'DeniedPayer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '027330C1-DCB1-4CCE-8D3D-6D4DC325CFAF', 'max': 'FC574DD7-0F1E-4350-A8F8-A5F7CB490D60', 'missing': '0', 'distinct': '44'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DeniedPayer'}, {'feature_label': 'DeniedPayer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DeniedPayer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PaidPayer', 'transformation_label': 'String Indexer'}], 'feature': 'PaidPayer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '082CDC0C-D1D1-4E86-879F-532772ED618A', 'max': 'FC574DD7-0F1E-4350-A8F8-A5F7CB490D60', 'missing': '0', 'distinct': '45'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PaidPayer'}, {'feature_label': 'PaidPayer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PaidPayer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '28.65', 'stddev': '18.93', 'min': '109', 'max': '33', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'RemarkCode', 'transformation_label': 'String Indexer'}], 'feature': 'RemarkCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'CR26', 'max': 'PR31', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'RemarkCode'}, {'feature_label': 'RemarkCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('RemarkCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Benefits Max/Exh', 'max': 'Eligibility', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Charges are covered under a capitation agreement/managed care plan.', 'max': 'This injury/illness is the liability of the no-fault carrier.', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Remarkcode1', 'transformation_label': 'String Indexer'}], 'feature': 'Remarkcode1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'CO109', 'max': 'PR33', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Remarkcode1'}, {'feature_label': 'Remarkcode1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Remarkcode1')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run rrxgbdsmotephaniallranksHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT\r\n\r\ne.EncounterID,\r\nT.TransactionDate, T.DateofService,Tr.ReasonDate,\r\n\r\nFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\nCASE WHEN bi.EncounterID = E.EncounterID AND bi.BillId = t.BillId THEN bi.PayerID END as BilledPayer,\r\nCASE WHEN trc.TransactionReasonCodeID = tr.TransactionReasonCodeID AND trc.ReasonType = 'Denial' THEN tr.PayerID END as DeniedPayer,\r\nCASE WHEN ((t.TransactionTypeID =2 AND t.Amount >=0 ) OR (t.TransactionTypeID =1)) and T.TransactionDate >= tsk.CompletedDate THEN t.PayerID END as PaidPayer,\r\n\r\ntrc.Reasoncode,\r\nrc.SourceCode as RemarkCode,\r\ntrc.ReasonCategory,\r\ntrc.ReasonDescription,\r\ntr.SourceKey as Remarkcode1,\r\n\r\nCASE\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 6 THEN 6\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 8 THEN 8\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 13 THEN 13\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 16 THEN 16\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 41 THEN 41\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 42 THEN 42\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 43 THEN 43\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 48 THEN 48\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 49 THEN 49\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 61 THEN 61\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 63 THEN 63\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 64 THEN 64\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 65 THEN 65\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 79 THEN 79\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 80 THEN 80\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 81 THEN 81\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 82 THEN 82\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 83 THEN 83\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 85 THEN 85\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 89 THEN 89\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 789 THEN 789\r\nWHEN ((t.transactiontypeid = 2 and t.Amount >= 0) OR (t.transactiontypeid =1 )) and tcr.TaskCompletionReasonID = 30899 THEN 30899\r\n\r\nELSE '0' END AS label_resolution\r\n\r\nFROM\r\nTransactions t \r\ninner join Transactiontype tt on tt.transactiontypeid = t.transactiontypeid \r\nInner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailId = t.ChargeDetailId AND T.TransactionDate>=TR.ReasonDate \r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\nleft join ChargeDetail cd on cd.ChargeDetailId = t.ChargeDetailId\r\nleft join Bill bi on bi.BillId = t.BillId\r\nleft join Encounter e on e.encounterid = bi.encounterid\r\nleft outer join ReasonCodeMapping rc on rc.RealCodeID=trc.TransactionReasonCodeID \r\n\t\t\t\tand  (\r\n\t\t\t\t\t\t(trc.ReasonCode IN ('22','33') \r\n\t\t\t\t\t\tand (rc.SourceCode like'M%' or rc.SourceCode like'N%') ) \r\n\t\t\t\t\t\tOR (trc.ReasonCode NOT IN ('22','33') \r\n\t\t\t\t\t\tand (rc.SourceCode like'CR%'or rc.SourceCode like'PR%')) \r\n\t\t\t\t      ) \r\nleft outer join Task tsk WITH (NOLOCK) on e.encounterid = tsk.encounterid and e.facilityid = tsk.facilityid and tsk.createddate >= tr.reasondate\r\nleft outer join TaskCompletionReason tcr WITH (NOLOCK) on tsk.TaskCompletionReasonID = tcr.TaskCompletionReasonID\r\nleft outer join Taskstatus ts on tsk.TaskStatusID=ts.TaskStatusID\r\nleft outer join TaskType tskt on tsk.TaskTypeID=tskt.TaskTypeID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = e.PatientID\r\n\r\nWHERE tr.ReasonDate between '2018-01-01' and '2020-12-31'\r\nand tsk.CreatedDate >= tr.ReasonDate\r\nand T.TransactionDate >= tsk.CompletedDate\r\nand t.transactiontypeid IN (1,2,4) \r\nand bi.IsBilled=1\r\nAND tr.Amount >.01\r\nand trc.reasonType = 'Denial'\r\nand trc.ReasonCode IN ('19','20','21','22','24','26','27','28','30','31','32','33','109','166','200')\r\nand ts.TaskStatusCategory ='Completed'\r\nand tskt.TaskTypeCategory='AR'\r\nand tcr.TaskCompletionReasonID NOT IN ('99','100','101')\r\nand tcr.TaskCompletionReason !='NULL'\r\nand tsk.TaskCompletionReasonID NOT IN ('44','62')", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run rrxgbdsmotephaniallranksHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "EncounterID", "transformation_label": "String Indexer"}], "feature": "EncounterID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0001A61A-F158-4D97-85D4-E547EE784C45", "max": "FFD4FF35-FCA5-4FB3-BE27-2595F30416F7", "missing": "0", "distinct": "485"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "EncounterID"}, {"transformationsData": [{"feature_label": "TransactionDate", "transformation_label": "Extract Date"}], "feature": "TransactionDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "TransactionDate"}, {"transformationsData": [{"feature_label": "DateofService", "transformation_label": "Extract Date"}], "feature": "DateofService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateofService"}, {"transformationsData": [{"feature_label": "ReasonDate", "transformation_label": "Extract Date"}], "feature": "ReasonDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ReasonDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "70.2", "stddev": "16.85", "min": "4", "max": "103", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "BilledPayer", "transformation_label": "String Indexer"}], "feature": "BilledPayer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "057B6691-F37E-4254-BCE1-D725BDCCD39C", "max": "FC574DD7-0F1E-4350-A8F8-A5F7CB490D60", "missing": "0", "distinct": "49"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BilledPayer"}, {"transformationsData": [{"feature_label": "DeniedPayer", "transformation_label": "String Indexer"}], "feature": "DeniedPayer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "027330C1-DCB1-4CCE-8D3D-6D4DC325CFAF", "max": "FC574DD7-0F1E-4350-A8F8-A5F7CB490D60", "missing": "0", "distinct": "44"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DeniedPayer"}, {"transformationsData": [{"feature_label": "PaidPayer", "transformation_label": "String Indexer"}], "feature": "PaidPayer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "082CDC0C-D1D1-4E86-879F-532772ED618A", "max": "FC574DD7-0F1E-4350-A8F8-A5F7CB490D60", "missing": "0", "distinct": "45"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PaidPayer"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "28.65", "stddev": "18.93", "min": "109", "max": "33", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "RemarkCode", "transformation_label": "String Indexer"}], "feature": "RemarkCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "CR26", "max": "PR31", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "RemarkCode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Benefits Max/Exh", "max": "Eligibility", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Charges are covered under a capitation agreement/managed care plan.", "max": "This injury/illness is the liability of the no-fault carrier.", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"feature_label": "Remarkcode1", "transformation_label": "String Indexer"}], "feature": "Remarkcode1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "CO109", "max": "PR33", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Remarkcode1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label_resolution", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "25.71", "stddev": "29.11", "min": "0", "max": "65", "missing": "0"}, "updatedLabel": "label_resolution"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run rrxgbdsmotephaniallranksHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(autofe, ["Age", "EncounterID_stringindexer", "TransactionDate_dayofmonth", "TransactionDate_month", "TransactionDate_year", "DateofService_dayofmonth", "DateofService_month", "DateofService_year", "ReasonDate_dayofmonth", "ReasonDate_month", "ReasonDate_year", "Gender_stringindexer", "BilledPayer_stringindexer", "DeniedPayer_stringindexer", "PaidPayer_stringindexer", "Reasoncode_stringindexer", "RemarkCode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer", "Remarkcode1_stringindexer"], "label_resolution")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
